In [27]:
import pandas as pd
from skbio.stats.ordination import OrdinationResults

# Für später
# from qiime2 import Metadata
# from qiime2 import Artifact
# from qiime2.plugins import emperor
# from qiime2.plugins import diversity


1: Dataframe erstellen mit metadata.tsv datei

1.1: Einlesen des Metadata file

In [28]:
df1 = pd.read_csv('/home/sven/Desktop/bsc_env/qp-platemapper/platemapper/tests/data/Moeller_SANS1/meta_plate.tsv' , sep='\t')
#Path ist noch lokal, work in progress
df2 = df1 

1.2 Falls Metadata file einen anderen namen als well_id haben sollte dann namen finden und umbenennen, anschließend checken ob well_id teil der indizes ist.
okay also problem, sobald die metadata file bereits den spaltennamen "well_id" enthält will die conversion nicht mehr funktionieren. Auch der untere Code bekommt es (noch) nicht hin.

In [29]:
werte = ["A1", "A2"] # Das hier muss noch unbedingt mit regex gemacht werden für den wahrscheinlichen fall dass auf der 
#plate die ersten beiden wells nicht benutzt werden.


# Passende Spalte finden und umbenennen
if "well_id" not in df2.columns:
    for spalte in df2.columns:
        if all(wert in df2[spalte].values for wert in werte):
            print(f"beide Werte in spalte: {spalte}")
else:
    1
   
df2 = df2.rename(columns={spalte: "well_id"}) 
# Andernfalls eine variable mit den wellids erstellen und damit weiterarbeiten

# testing: nachschauen ob es geklappt hat
if "well_id" in df2.columns:
    print("ja")

ja


Faxen (also wirklich nur faxen, es gibt eine so viel bessere idee und die werde ich dann auch machen allerdings funktioniert das hier (leider gottes) und es erlaubt mir die weiteren schritte zu gehen.)

In [30]:

df2["well_id"] = df2["well_id"].str.replace("A", "8, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("B", "7, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("C", "6, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("D", "5, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("E", "4, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("F", "3, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("G", "2, ", regex=False)
df2["well_id"] = df2["well_id"].str.replace("H", "1, ", regex=False)

1.3: Das große Metadata file verkleinern in die für die Ordination interessanten Informationen(sample_name, well_id) herausfiltern

In [31]:
tempSamples = pd.concat([
    df2["sample_name"],
    df2["well_id"]],
    axis =1 
    )

# Sehr wichtig: Es kann passieren dass in sample_name mehr rows 
# existieren als well_id werte, dann packt der code fälschlicherweise NaN values auf die row.
# Hiermit versucht man dem ein bisschen entgegenzuwirken und so far funktioniert es
x = 0
for val in tempSamples["well_id"].values:
    x += 1
    if type(val) is not str:
        print(type(val)) # NaN ist n float
        print(tempSamples.sample_name.values[x-1])
        tempSamples = tempSamples.drop(x - 1)

# testing: worked?
print(tempSamples.tail(5))


<class 'float'>
time_point
                  sample_name well_id
144              14908.dlr145    4, 3
145  14908.dlrnegcontrolK5Y8C   1, 12
146  14908.dlrposcontrolK5Y8C   1, 11
147  14908.dlrnegcontrolK65YY   1, 12
148  14908.dlrposcontrolK65YY   1, 11


Ab hier, Chatgpt war die inspiration, nach kurzem googlen hab ich herausgefunden wie ich das für mich umschreiben muss. 

1.4: Die zusammengesetzten Zahlen der well_id trennen und in zwei spalten (Axis1, Axis2) trennen. Zum ende hin die nummern als float ausgeben lassen da es mit str vermutlich noch weniger funktioniert. 

In [32]:
#split
tempSamples[["Axis1", "Axis2"]] = tempSamples["well_id"].str.split(',', n=1, expand=True)

# dabei schon die Achsen tauschen, da:
#     x -------->
#         0 1 2 3 ...
# y   a 
# |   b
# |   c
# |
# Im Originalzustand enthält Axis1 die a,b,c... werte und Axis2 die 0-12 werte, die ordination datei nimmt als erste achse halt x
# Ich brauche noch einen mechanismus welcher bei bspw. 1A als wellid die achsen nicht dreht

#"Drehen" der beiden Achsen
# datentypen fest deklarieren, Ordination kann nicht mit str arbeiten, muss/kann float sein
temp = tempSamples["Axis1"]
tempSamples["Axis1"] = tempSamples["Axis2"].astype(float)
tempSamples["Axis2"] = temp.astype(float)

#finalen dataframe erstellen aus welchem auch die ordination am ende entsteht
samples = pd.DataFrame(
    data=tempSamples[["Axis1", "Axis2"]].values,
    index=tempSamples["sample_name"],
    columns=["Axis1", "Axis2"]
)
#test
print(samples.head(7))

             Axis1  Axis2
sample_name              
14908.dlr1     1.0    8.0
14908.dlr2     2.0    8.0
14908.dlr3     3.0    8.0
14908.dlr4     4.0    8.0
14908.dlr5     5.0    8.0
14908.dlr6     6.0    8.0
14908.dlr7     7.0    8.0


2.1: Eigenvalues und proportion_explained. Einfach fantasiewerte einfügen

In [33]:
eigvals = pd.Series([0.13, 0.37])
proportion_explained = pd.Series([0.5758, 0.4242])


2.2: Erstellung der Ordination und export als ordination.txt

In [34]:
ordination = OrdinationResults(
    "Manual",
    "Parsed well ID ordination",
    eigvals,
    samples,
    proportion_explained = proportion_explained
    # fun fact wenn man proportion_explained im selben stil wie samples & eigvals schreibt sprengt das komplett die Ordiantion
)


In [35]:
with open("ordination.txt", "w") as f:
    
    ordination.write(f, format="ordination")

Nächster Teil: mit Python automatisch einen Path und Ordner für das alles erstellen und dann darüber automatisch die .qzv zu erstellen.
Probleme die ich immernoch habe:    
- Der Code kann nur eine plate erstellen(falls mehr koordinaten als plätze auf wells, und dann auch noch gleich, dann bomba)
    - vielleicht dem Programm sagen, sobald mehr als 96 wellids da sind dann pack die in einen dateframe und geh weitere 96 schritte AUßER wir haben etabliert es handelt sich um eine 384x plate
    - ODER: bei mehreren gleichen well_ids nach irgendeinem Alleinstellungsmerkmal(krank,gesund) filtern und danach mehrere
    plates erzeugen.
- 384x plates sind noch nicht möglich (probably easiest to do)
    - Vielleicht irgendwie es schaffen, dem code zu sagen wie viele wells in einem Metafile stecken, demnach in mehrere Dataframes unterteilen
    - in etwa so: for vals in df2["Axis1"] if df2["Axis1"].values >=13 && df2["Axis2"].values >=9 dann 384 well plate
- Qiime2 Integration in den Code, skip the Terminal